# Tổng quan vấn đề

* Một vấn đề tồn tại với bất kì trang web lớn nào hiện nay là làm thế nào xử lý nội dung độc hại. Quora muốn giải quyết vấn đề này trực tiếp để giữ cho nền tảng của họ trở thành một nơi mà người dùng có thể cảm thấy an toàn khi chia sẻ kiến thức của họ với thế giới.
* Quora là một nền tảng cho phép mọi người học hỏi lẫn nhau. Trên Quora, mọi người có thể đặt câu hỏi và kết nối với những người khác, những người đóng góp thông tin chi tiết độc đáo và câu trả lời chất lượng. Một thách thức quan trọng là loại bỏ những câu hỏi insincere - những câu hỏi được đặt ra dựa trên những tiền đề sai lầm hoặc có ý định đưa ra một tuyên bố hơn là tìm kiếm câu trả lời hữu ích.
* Mục tiêu: loại bỏ những câu hỏi insincere


In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import math

In [ ]:
train = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
test = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')

In [ ]:
train.info()

+ Tập dữ liệu train gôm 13 triệu dòng và 3 cột
+ Các trường dữ liệu:
 * qid: mã định danh
 * question_text: các câu hỏi trên quora
 * target: câu hỏi sincere khi có giá trị 0 và insincere khi có giá trị 1

In [ ]:
train.head(10)

In [ ]:
train["target"].value_counts()

Tập dữ liệu train gồm 80810 dòng đã được xác nhận là câu hỏi insincere và 1225312 dòng là sincere

In [ ]:
test.info()

Tập dữ liệu test gồm hơn 300 nghìn dòng

In [ ]:
test.head(10)

In [ ]:
fig = plt.figure(figsize=(5,3))
ax = fig.add_axes([0,0,2,1])
type_check = ['sincere','insincere']
count = [1225312,80810]
ax.bar(type_check,count)
plt.show()

In [ ]:
print('- Phần trăm câu hỏi sincere (target = 0): {}%'.format(100 - round(train['target'].mean() * 100, 2))) 
print('- Phần trăm câu hỏi insincere (target = 1): {}%'.format(round(train['target'].mean() * 100, 2)))

* Câu hỏi sincere chiếm 93,81% trong khi câu hỏi insincere chỉ chiếm 6,19% trên tập dữ liệu train
* Như vậy, tập dữ liệu của chúng ta rất không cân bằng
* Giải pháp: làm cho mô hình cân bằng hơn sao cho không ảnh hưởng đáng kể đến khả năng dự báo của mô hình

In [ ]:
# chia lại tập dữ liệu theo tỉ lệ 4:1
from sklearn.utils import resample
sincere = train[train.target == 0]
insincere = train[train.target == 1]
train_sample = pd.concat([resample(sincere,replace = True,n_samples = len(insincere)*4), 
insincere])
train_sample

# Xử lý dữ liệu

In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[',
          ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', '·', '_', '{', '}',
          '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′',
          'Â', '█', '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬',
          '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒',
          '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦',
          '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»',
          '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤',
          '‡', '√' ]

In [ ]:
# Loại bỏ các kí tự đặc biệt
def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

In [ ]:
# Loại bỏ số
def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

In [ ]:
mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}

In [ ]:
# Thay thế các từ viết tắt
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [ ]:
# Loại bỏ các từ trong stopword
import nltk
from nltk.corpus import stopwords
stopword_list = nltk.corpus.stopwords.words('english')
def remove_stopwords(text, is_lower_case=True):
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [ ]:
# Stemming
from nltk.stem import SnowballStemmer
from nltk.tokenize.toktok import ToktokTokenizer
def stem_text(text):
    tokenizer = ToktokTokenizer()
    stemmer = SnowballStemmer('english')
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

Stemming là kỹ thuật dùng để biến đổi 1 từ về dạng gốc (được gọi là stem hoặc root form) bằng cách cực kỳ đơn giản là loại bỏ 1 số ký tự nằm ở cuối từ mà nó nghĩ rằng là biến thể của từ

In [ ]:
# Lemmatization
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.toktok import ToktokTokenizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemma_text(text):
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    tokens = [wordnet_lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

Lemmatization khác với Stemming là xử lý bằng cách loại bỏ các ký tự cuối từ một cách rất heuristic, Lemmatization sẽ xử lý thông minh hơn bằng một bộ từ điển hoặc một bộ ontology nào đó

In [ ]:
def clean_sentence(x):
    x = x.lower()
    x = clean_text(x)
    x = clean_numbers(x)
    x = replace_typical_misspell(x)
    x = remove_stopwords(x)
    x = stem_text(x)
    x = lemma_text(x)
    x = x.replace("'","")
    return x

In [ ]:
# word cloud cho câu hỏi sincere
from wordcloud import WordCloud, STOPWORDS
stop_words = set(STOPWORDS)
sincere_wordcloud = WordCloud().generate(str(train[train["target"] == 0]["question_text"]))
plt.figure(figsize=(8,7))
plt.imshow(sincere_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
# word cloud cho câu hỏi insincere
insincere_wordcloud = WordCloud().generate(str(train[train["target"] == 1]["question_text"]))
plt.figure(figsize=(8,7))
plt.imshow(insincere_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
#Xử lí dữ liệu trên cả tập train và tập test 
train_sample['question_text'] = train_sample['question_text'].apply(lambda x: clean_sentence(x))
test['question_text'] = test['question_text'].apply(lambda x: clean_sentence(x))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import normalized_mutual_info_score
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

X_train, X_test, y_train, y_test = train_test_split(train_sample['question_text'], 
train_sample['target'], test_size=0.3)

# Bag of Words (BoW)
* Với một văn bản thì feature vector sẽ có dạng như thế nào? Làm sao đưa các từ, các câu, đoạn văn ở dạng text trong các văn bản về một vector mà mỗi phần tử là một số?
* Có một phương pháp rất phổ biến giúp bạn trả lời những câu hỏi này. Phương pháp đó có tên là Bag of Words (BoW) (Túi đựng Từ)
* Nhược điểm: không mang thông tin về thứ tự của các từ; cũng như sự liên kết giữa các câu, các đoạn văn trong văn bản
* CountVectorizer để chuyển đổi văn bản thành một vector

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(list(X_train) + list(X_test))
X_train = vectorizer.transform(X_train) 
X_test = vectorizer.transform(X_test)

# Logistic Regression

Phương pháp logistic regression là một mô hình hồi quy nhằm dự đoán giá trị đầu ra rời rạc (discrete target variable) y ứng với một véc-tơ đầu vào x. Việc này tương đương với chuyện phân loại các đầu vào x vào các nhóm y tương ứng


In [ ]:
logistic = LogisticRegression() 
logistic.fit(X_train,y_train)

In [ ]:
train_predictions = logistic.predict(X_train)
train_acc = accuracy_score(y_train, train_predictions)  
train_f1 = f1_score(y_train, train_predictions) 
print(f"Training accuracy: {train_acc:.2%}, F1: {train_f1:.4f}") 
test_predictions = logistic.predict(X_test)
test_acc = accuracy_score(y_test, test_predictions) 
test_f1 = f1_score(y_test, test_predictions) 
print(f"Testing accuracy:  {test_acc:.2%}, F1: {test_f1:.4f}")

F1_score xấp xỉ 0,73

# Result

In [ ]:
x_val = vectorizer.transform(test['question_text'])
val_predictions = logistic.predict(x_val)
test['prediction'] = val_predictions
submission = test[['qid', 'prediction']]
submission.to_csv('submission.csv', index=False)
submission